In [ ]:
location_data = [
['Centrico Plaza', 'Gare de Lumiose', 'Pokemon Research Lab', 'Hotel Z', 'Racine Construction', 'Restaurant Le Nah', 'Rust Syndicate Office', 'Lumiose Sewers (Canal Access)', 'Quasartico Inc.', 'Lysandre Cafe', 'Lumiose Sewers (Main Access)', 'Hotel Richissime', 'Looker Bureau', 'Lumiose Museum', 'Restaurant Le Yeah', 'Sushi High Roller', 'Restaurant Le Wow', 'Justice Dojo', 'Vert Pokemon Center', 'Bleu Pokemon Center', 'Vernal Pokemon Center', 'Magenta Pokemon Center', 'Magenta Plaza Pokemon Center', 'Rouge Pokemon Center', 'Centrico Pokemon Center', 'Jaune Pokemon Center', 'Hibernal Pokemon Center', 'Cafe Cyclone', 'Cafe Classe', 'Cafe Introversion', 'Nouveau Cafe', 'Cafe Woof', 'Cafe Soleil', 'Shutterbug Cafe', 'Nouveau Cafe Truck No. 2', 'Cafe Pokemon-Amie', 'Cafe Rouleau', 'Cafe Gallant', 'Cafe Triste', 'Nouveau Cafe Truck No. 3', 'Cafe Ultimo', 'Cafe Action', 'Cafe Kizuna', 'Cafe Bataille', 'Wild Zone 1', 'Wild Zone 2', 'Wild Zone 3', 'Wild Zone 4', 'Wild Zone 5', 'Wild Zone 6', 'Wild Zone 7', 'Wild Zone 8', 'Wild Zone 9', 'Wild Zone 10', 'Wild Zone 11', 'Wild Zone 12', 'Wild Zone 13', 'Wild Zone 14', 'Wild Zone 15', 'Wild Zone 16', 'Wild Zone 17', 'Wild Zone 18', 'Wild Zone 19', 'Wild Zone 20'],
['All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'All Locations', 'Facilities', 'Facilities', 'Facilities', 'All Locations', 'All Locations', 'All Locations', 'Facilities', 'Facilities', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Pokemon Centers', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Cafes', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones', 'Wild Zones'],
[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, -2], [1, -1], [1, 0], [1, 1], [1, 2], [1, 0], [2, 0], [2, 1], [2, -1], [2, 0], [2, 1], [-1, -1], [-1, 0], [0, 0], [0, 1], [1, 0], [1, 1], [1, 2], [-2, -1], [-2, 0], [-1, -1], [-1, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, -2], [1, -1], [1, 0], [1, 1], [-2, 0], [2, 0], [2, 1], [2, 2], [-1, -3], [-1, -2], [-1, -1], [-1, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, -2], [1, -1], [1, 0], [1, 1], [1, 2], [1, 3], [2, -2], [2, -1], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6]]
]

In [ ]:
locations_in_file = [
["All travel spots"],
["Facilities",
"Centrico Plaza", "Gare de Lumiose", "Pokemon Research Lab", "Hotel Z", "Racine Construction", "Restaurant Le Nah", "Rust Syndicate Office",
"Lumiose Sewers (Canal Access)", "Quasartico Inc.", "Lysandre Cafe", "Lumiose Sewers (Main Access)", "Hotel Richissime", "Looker Bureau", "Lumiose Museum",
"Restaurant Le Yeah", "Sushi High Roller", "Restaurant Le Wow", "Justice Dojo"],
["Pokemon Centers",
"Vert Pokemon Center", "Bleu Pokemon Center", "Vernal Pokemon Center", "Magenta Pokemon Center", "Magenta Plaza Pokemon Center", "Rouge Pokemon Center", "Centrico Pokemon Center",
"Jaune Pokemon Center", "Hibernal Pokemon Center"],
["Cafes",
"Cafe Cyclone", "Cafe Classe", "Cafe Introversion", "Nouveau Cafe", "Cafe Woof", "Cafe Soleil", "Shutterbug Cafe",
"Nouveau Cafe Truck No. 2", "Cafe Pokemon-Amie", "Cafe Rouleau", "Cafe Gallant", "Cafe Triste", "Nouveau Cafe Truck No. 3", "Cafe Ultimo",
"Cafe Action", "Cafe Kizuna", "Cafe Bataille"],
["Wild Zones",
"Wild Zone 1", "Wild Zone 2", "Wild Zone 3", "Wild Zone 4", "Wild Zone 5", "Wild Zone 6", "Wild Zone 7",
"Wild Zone 8", "Wild Zone 9", "Wild Zone 10", "Wild Zone 11", "Wild Zone 12", "Wild Zone 13", "Wild Zone 14",
"Wild Zone 15", "Wild Zone 16", "Wild Zone 17", "Wild Zone 18", "Wild Zone 19", "Wild Zone 20"]
]

In [ ]:
def convert_to_single_array():
    output_array = []
    for array in locations_in_file:
        for item in array[1:]:
            output_array.append(item)
    return output_array

In [ ]:
def get_categories():
    new_array = ["All Locations"]
    for array in locations_in_file:
        new_array.append(array[0])
    return new_array

In [12]:
category = get_categories()

In [13]:
for i in range(len(full_coordinates)):
    test_category = location_data[1][i]
    print("Item:", i)
    print("Test category:", test_category)
    print("Category index:", categories.index(test_category))
    print("Category Coordinate:", full_coordinates[i][0])
    assert(categories.index(test_category) == full_coordinates[i][0])

Item: 0
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 1
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 2
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 3
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 4
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 5
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 6
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 7
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 8
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 9
Test category: All Locations
Category index: 0
Category Coordinate: 0
Item: 10
Test category: Facilities
Category index: 1
Category Coordinate: 0


AssertionError: 

In [ ]:

def append_coordinates(index):
    for category in location_data[1]:
        j = 0
        while (category == categories[j] and j < 5):
            j += 1
    new_array = []
    new_array.append(j)
    new_array.append(location_data[2][index][0])
    new_array.append(location_data[2][index][1])
    return new_array

full_coordinates = []
for i in range(len(location_data[0])):
    full_coordinates.append(append_coordinates(i))

print(full_coordinates)